## 4 - Disabling Gradient Tracking Temporarily

In [1]:
import torch, torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

When training neural networks, automatic differentiation module of PyTorch essential for backpropagation. For other purposes, however, it has no use at all. For example, when testing the accuracy of the network on a test set, or visualizing the weights of some layer in the network, gradients are not used. Since tracking gradients requires caching the activations in the memory, it costs both memory and time. A common practice is to "temporarily" disable automatic differentiation whenever it not used. In `torch.nn.Module` gradient tracking is on by default while manually created tensors with functions such as `torch.randn` or `torch.zeros` are not differentiated automatically (remember from Lab 1).

In [2]:
#create a simple network

class Network(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Linear(200, 100)
        self.layer2 = torch.nn.Linear(100, 5)
    
    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        x = self.layer2(x)
        return x
    
network = Network()
print(network)

Network(
  (layer1): Linear(in_features=200, out_features=100, bias=True)
  (layer2): Linear(in_features=100, out_features=5, bias=True)
)


In [3]:
#print the bias tensor of layer 2
print(network.layer2.bias)

Parameter containing:
tensor([-0.0081,  0.0316,  0.0057,  0.0051, -0.0594], requires_grad=True)


In [4]:
# Make a forward pass with a dummy input and define loss as the mean of outputs
dummy_input = torch.randn(4, 200)
dummy_output = network(dummy_input)
dummy_loss = dummy_output.mean()
print(f'Input Shape: {dummy_input.shape}')
print(f'Output Shape: {dummy_output.shape}')
print(f'Loss: {dummy_loss}')

Input Shape: torch.Size([4, 200])
Output Shape: torch.Size([4, 5])
Loss: 0.046225838363170624


In [5]:
# Check the gradients of bias tensor of layer 2, before and after the backward operation
print(f'Gradients before backward: {network.layer2.bias.grad}')

dummy_loss.backward() 
#  Note: backward can be called  only once on a tensor since torch
# clears intermediate results after gradients are computed with .backward()
# so if you run this cell more than once, it gives an error.
# You can compute dummy_loss again run this cell again to clear the error message.

print(f'Gradients after backward: {network.layer2.bias.grad}')




Gradients before backward: None
Gradients after backward: tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])


---

PyTorch has `torch.no_grad()` to use with `with` block of Python to disable gradients in (and only in) that block. 

`with` keyword is a Python keyword. Its behaviour can be summarized as following;

"Using `with` keyword with a statement creates a with block and the statement used with `with` is only applied within the `with` block."

So that means whenever you are out of the block, the statement loses its effect.

In the example below `torch.no_grad()` effects the indented lines only.


In [6]:

with torch.no_grad():
    # these lines are executed in the effect of torch.no_grad()
    # which means their "requires_grad" field is set False automatically and torch
    # does not bother saving their outputs and "grad_fn" to the memory for calculating
    # gradients when you call .backward() somewhere.
    
    
    dummy_input_nograd = torch.randn(4, 200)
    dummy_output_nograd = network(dummy_input)
    dummy_loss_nograd = dummy_output.mean()

# Out of the with block,
# these lines are executed normally.
# So you can call .backward() on dummy loss, but not on dummy_loss_nograd

dummy_input = torch.randn(4, 200)
dummy_output = network(dummy_input)
dummy_loss = dummy_output.mean()

dummy_loss.backward() # It succesfully executes backward


In [7]:
dummy_loss_nograd.backward() 
# This variable is created in "with torch.no_grad()" block, so it does not have a 'grad_fn'
# so calling backward on dummy_loss_nograd is not possible.

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

TL;DR:
Whenever you don't need gradients, you should use `no_grad` block to save memory and speed.